
[Dataset](https://drive.google.com/file/d/1wWsrN2Ep7x6lWqOXfr4rpKGYrJhWc8z7/view)

In [ ]:
/content/drive/MyDrive/Colab Notebooks/speech-emotion-recognition-ravdess-data

In [2]:
import os
Root = "/content/drive/MyDrive/Colab Notebooks/speech-emotion-recognition-ravdess-data"
os.chdir(Root)

In [3]:
ls

Actor_01/  Actor_06/  Actor_11/  Actor_16/  Actor_21/
Actor_02/  Actor_07/  Actor_12/  Actor_17/  Actor_22/
Actor_03/  Actor_08/  Actor_13/  Actor_18/  Actor_23/
Actor_04/  Actor_09/  Actor_14/  Actor_19/  Actor_24/
Actor_05/  Actor_10/  Actor_15/  Actor_20/  modelForPrediction1.sav


In [4]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [9]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [103]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['sad', 'fearful', 'calm', 'happy']

In [77]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/Colab Notebooks/speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [104]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [79]:
x_train

array([[-5.17965454e+02,  3.71398888e+01, -1.56945620e+01, ...,
         1.15746152e-04,  8.88544237e-05,  4.30482214e-05],
       [-6.35324280e+02,  5.15364532e+01,  2.86708784e+00, ...,
         1.64853787e-04,  1.09570530e-04,  8.09854173e-05],
       [-5.59538025e+02,  5.10846329e+01, -5.12032223e+00, ...,
         5.24717798e-05,  3.66673994e-05,  1.29582568e-05],
       ...,
       [-5.35107422e+02,  3.43586006e+01, -7.32960510e+00, ...,
         1.08357584e-04,  5.78423569e-05,  4.58427166e-05],
       [-5.17661743e+02,  4.90717125e+01, -7.92283487e+00, ...,
         5.29468052e-05,  4.08347332e-05,  1.58430666e-05],
       [-6.01385986e+02,  6.33781929e+01, -1.15760441e+01, ...,
         7.30556485e-06,  4.64438199e-06,  1.78243295e-06]])

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [105]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [106]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [123]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [127]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [128]:
#Predict for the test set
y_pred=model.predict(x_test)

In [85]:
y_pred

array(['calm', 'fearful', 'sad', 'disgust', 'disgust', 'fearful',
       'fearful', 'angry', 'happy', 'calm', 'fearful', 'angry', 'disgust',
       'fearful', 'fearful', 'disgust', 'fearful', 'angry', 'sad', 'sad',
       'angry', 'angry', 'happy', 'calm', 'sad', 'happy', 'sad', 'happy',
       'disgust', 'sad', 'sad', 'calm', 'happy', 'angry', 'angry',
       'angry', 'calm', 'fearful', 'happy', 'happy', 'fearful', 'happy',
       'sad', 'angry', 'happy', 'sad', 'fearful', 'happy', 'sad', 'calm',
       'fearful', 'disgust', 'fearful', 'fearful', 'disgust', 'fearful',
       'calm', 'sad', 'angry', 'happy', 'angry', 'angry', 'sad',
       'fearful', 'fearful', 'fearful', 'fearful', 'angry', 'sad',
       'fearful', 'sad', 'angry', 'angry', 'disgust', 'happy', 'fearful',
       'sad', 'sad', 'fearful', 'fearful', 'calm', 'disgust', 'disgust',
       'disgust', 'calm', 'calm', 'disgust', 'fearful', 'sad', 'fearful',
       'fearful', 'angry', 'angry', 'disgust', 'calm', 'angry', 'fearfu

In [129]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 64.06%


In [19]:
from sklearn.metrics import accuracy_score, f1_score

In [130]:
f1_score(y_test, y_pred,average=None)

array([0.60465116, 0.71559633, 0.65753425, 0.5862069 ])

In [131]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,fearful,fearful
1,happy,happy
2,sad,fearful
3,sad,sad
4,sad,sad
5,sad,sad
6,calm,fearful
7,happy,sad
8,happy,sad
9,sad,fearful


In [132]:
import pickle
# Writing different model files to file
with open( '/content/modelForPrediction_final.sav', 'wb') as f:
    pickle.dump(model,f)

In [133]:
filename = '/content/modelForPrediction_final.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/drive/MyDrive/Colab Notebooks/speech-emotion-recognition-ravdess-data/Actor_01/03-01-01-01-01-02-01.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['fearful'], dtype='<U7')

In [24]:
feature

array([[-6.93497009e+02,  5.00643921e+01,  5.71450531e-01,
         1.43299656e+01,  3.33637071e+00, -2.54071975e+00,
        -4.05790949e+00, -1.07119999e+01, -7.29413939e+00,
         1.74018872e+00, -4.19064236e+00,  1.95466173e+00,
        -5.24789381e+00,  2.78142977e+00, -3.16756773e+00,
        -3.40008307e+00, -2.37803197e+00, -5.68717539e-01,
        -6.47753334e+00, -1.24320555e+00, -2.80542517e+00,
        -5.43635798e+00, -4.46875393e-01, -3.63516593e+00,
        -2.98372602e+00, -5.63902557e-01, -1.65101993e+00,
        -5.55944741e-01, -3.41018438e+00, -2.24465466e+00,
        -3.13058877e+00, -2.70089960e+00, -1.88821292e+00,
        -5.54154456e-01, -3.96459913e+00, -2.13485193e+00,
        -3.94577074e+00, -1.62457860e+00, -2.03990698e+00,
        -3.62910867e+00,  6.33734703e-01,  6.48760676e-01,
         6.21744275e-01,  6.34553194e-01,  6.60345614e-01,
         6.60463989e-01,  7.00949013e-01,  7.31556356e-01,
         7.46921539e-01,  7.23420441e-01,  7.16119647e-0